In [1]:
import cassio

In [3]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] ="https://e6debf57-dc97-44b5-b93c-74bfbf0abef9-us-east-2.apps.astra.datastax.com/"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = "AstraCS:KBzARFACrAqhapGPnqJykgup:6c4838ecd8f06921c7182c77415d8b88cc3370cf747ff16e776620afa40a0085"
# os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

In [4]:
from langchain_astradb import AstraDBVectorStore
# from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
import os

# Configure your embedding model and vector store
embedding = OllamaEmbeddings()
vstore = AstraDBVectorStore(
    collection_name="test",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

Astra vector store configured


In [5]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

/Users/adityasaikolluru/Desktop/Langchain/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 450/450 [00:00<00:00, 40588.29 examples/s]

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [6]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [7]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [9]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': '282ee618e15d4d57b8ef7646c0fffcd1', 'content': 'The worst of all deceptions is self-deception.', 'metadata': {'author': 'plato'}}, {'_id': 'd9ecd11f1c864f8586fe110b7f29c294', 'content': 'Being is. Being is in-itself. Being is what it is.', 'metadata': {'author': 'sartre', 'history': 'y', 'knowledge': 'y', 'ethics': 'y'}}, {'_id': '976b7f311c4c42488b5c1802fd71d2b9', 'content': 'The most spiritual men, as the strongest, find their happiness where others would find their destruction: in the labyrinth, in hardness against themselves and others, in experiments. Their joy is self-conquest: asceticism becomes in them nature, need, and instinct. Difficult tasks are a privilege to them; to play with burdens that crush others, a recreation. Knowledge-a form of asceticism. They are the most venerable kind of man: that does not preclude their being the most cheerful and the kindliest.', 'metadata': {'author': 'nietzsche', 'knowledge': 'y'}}, {'_id': 'a3ade8a4c2b34f9

In [ ]:
# Basic Retrieval
# Retrieve context from your vector database, and pass it to the model with a prompt.m\

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.llms import Ollama

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = Ollama(model="gemma")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what is the most important to allow the brain and provide me the tags?")

'The provided text does not contain any information regarding the importance of allowing the brain or any specific tags, so I am unable to answer this question from the given context.'

In [11]:
# WARNING: This will delete the collection and all documents in the collection
vstore.delete_collection()